# Model Predictive Control of Hybrid Systems

Given the state of the system at time zero $x(0)$, we solve the following (open-loop) optimal-control problem 
\begin{align}
V^*(x(0)) :=
\min_{x(\cdot), u(\cdot)} \quad &
\frac{1}{2} \sum_{t=0}^{N-1} x^T(t) Q x(t) + u^T(t) R u(t) + \frac{1}{2} x^T(N) P x(N),
\\
\text{subject to} \quad & x(t+1) = A_i x(t) + B_i u(t) + c_i
\quad \text{if} \quad
(x(t),u(t)) \in \mathcal D_i
,
\quad t=0,\ldots, N-1,
\\
& x(N) \in \mathcal X_N,
\end{align}
we apply the control $u(0)$ to the system, and we move to the next time step.

Assuming $c_1 = 0$ and $0 \in \mathrm{int} (D_1)$, we can ensure stability choosing $P$ as the solution of the infinite-horizon DARE for the system in mode $i = 1$, and $X_N$ as any constraint-admissible invariant set for the system in mode $1$ in closed loop with the LQR controller.

## Demo

In [1]:
# notebook settings
%load_ext autoreload
%autoreload 2

# external imports
import numpy as np
import matplotlib.pyplot as plt

# internal imports
from pympc.geometry.polyhedron import Polyhedron
from pympc.dynamics.discrete_time_systems import LinearSystem, AffineSystem, PieceWiseAffineSystem
from pympc.control.controllers import HybridModelPredictiveController
from pympc.plot import plot_input_sequence, plot_state_trajectory, plot_state_space_trajectory

We stabilize a linear inverted pendulum with mass $m$ and length $l$, subject to the gravity acceleration $g$; the pendulum is allowed to impact with an elastic wall (with stiffness $k$) placed at distance $d$ from the vertical line passing trough the hinge

In [2]:
m = 1.
l = 1.
g = 10.
k = 10000.
d = .1
h = .01

We express the linearized dynamics in PieceWise Affine (PWA) form
\begin{equation}
\dot x =
\begin{cases}
A_1 x + B_1 u, \quad \text{if} \quad (x, u) \in \mathcal D_1, \\
A_2 x + B_2 u + c_2, \quad \text{if} \quad (x, u) \in \mathcal D_2,
\end{cases}
\end{equation}

with

\begin{equation}
A_1 = 
\begin{bmatrix}
0 & 1 \\ g/l & 0
\end{bmatrix},
\quad
B_1 = 
\begin{bmatrix}
0 \\ 1/(ml^2)
\end{bmatrix},
\quad
A_2 = 
\begin{bmatrix}
0 & 1 \\ g/l - k/m & 0
\end{bmatrix},
\quad
B_2 = 
\begin{bmatrix}
0 \\ 1/(ml^2)
\end{bmatrix},
\quad
c_2 = 
\begin{bmatrix}
0 \\ kd/(ml)
\end{bmatrix},
\end{equation}

\begin{equation}
\mathcal D_1 =
\{ (x,u) \ | \
x_1 \leq d/l, \
x_{\mathrm{min}} \leq x \leq x_{\mathrm{max}}, \
u_{\mathrm{min}} \leq u \leq u_{\mathrm{max}} \},
\end{equation}

\begin{equation}
\mathcal D_2 =
\{ (x,u) \ |
\ x_1 > d/l, \
x_{\mathrm{min}} \leq x \leq x_{\mathrm{max}},
\ u_{\mathrm{min}} \leq u \leq u_{\mathrm{max}} \}
\end{equation}

In [3]:
# discretization method
method = 'explicit_euler'

# dynamics n.1
A_1 = np.array([[0., 1.],[g/l, 0.]])
B_1 = np.array([[0.],[1/(m*l**2.)]])
S_1 = LinearSystem.from_continuous(A_1, B_1, h, method)

# dynamics n.2
A_2 = np.array([[0., 1.],[g/l-k/m, 0.]])
B_2 = B_1
c_2 = np.array([[0.],[k*d/(m*l)]])
S_2 = AffineSystem.from_continuous(A_2, B_2, c_2, h, method)

# list of dynamics
S_list = [S_1, S_2]

The domains of the PWA must be expressed as (bounded!) polyhedra

In [4]:
# state domain n.1
x_min_1 = np.array([[-2.*d/l],[-1.5]])
x_max_1 = np.array([[d/l], [1.5]])
X_1 = Polyhedron.from_bounds(x_min_1, x_max_1)
assert X_1.bounded

# state domain n.2
x_min_2 = np.array([[d/l], [-1.5]])
x_max_2 = np.array([[2.*d/l],[1.5]])
X_2 = Polyhedron.from_bounds(x_min_2, x_max_2)
assert X_2.bounded

# input domain
u_min = np.array([[-4.]])
u_max = np.array([[4.]])
U = Polyhedron.from_bounds(u_min, u_max)
assert U.bounded

# domains
D_1 = X_1.cartesian_product(U)
D_2 = X_2.cartesian_product(U)
D_list = [D_1, D_2]

The overall PWA system can be defines as follows

In [10]:
S = PieceWiseAffineSystem(S_list, D_list)

We can now synthesize the controller

In [11]:
# controller parameters
N = 10
Q = np.eye(S.nx)
R = np.eye(S.nu)

# terminal set and cost
P, K = S_1.solve_dare(Q, R)
X_N = S_1.mcais(K, D_1)[0]

In [34]:
# hybrid MPC controller
controller = HybridModelPredictiveController(S, N, Q, R, P, X_N)

----------------------- We can now simulate the closed-loop dynamics starting from the initial state $x_0$.

In [ ]:
N_sim = 100
x_0 = np.array([[.0],[.79]])
u = []
x = [x_0]
for k in range(N_sim):
    print('Time step ' + str(k) + '.\r'),
    u.append(controller.feedback(x[k]))
    x_next = pwa_sys.simulate(x[k], [u[k]])[0][1]
    x.append(x_next)

We can use the plot functions to visualize the time evolution of the system.

In [ ]:
mpc_plt.input_sequence(u, t_s, (u_min, u_max))
plt.show()
mpc_plt.state_trajectory(x, t_s, (x_min_0, x_min_1, x_max_1))
plt.show()

In [ ]:
X_0.plot(facecolor='g')
X_1.plot(facecolor='r')
mpc_plt.state_space_trajectory(x, color='w')
plt.show()

In order to make computations faster, we can warm start the MIQP solver at each step time.

In [ ]:
u = []
x = [x_0]
u_ws = None
x_ws = None
ss_ws = None
terminal_mode = 0
for k in range(N_sim):
    u_k, x_k, ss_k = controller.feedforward(x[k], u_ws, x_ws, ss_ws)[0:3]
    x_next = pwa_sys.simulate(x[k], [u_k[0]])[0][1]
    u.append(u_k[0])
    x.append(x_next)
    u_ws = u_k[1:] + [K.dot(x_k[-1])]
    x_ws = x_k[1:] + [pwa_sys.simulate(x_k[-1], [u_ws[-1]])[0][1]]
    ss_ws = ss_k[1:] + (terminal_mode,)